<a href="https://colab.research.google.com/github/IRONMAN-AIcoder/genai/blob/main/creativeproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**"KnowLaw: AI-Powered Legal Document Assistant"**

Project Context:

"KnowLaw" is an AI-driven legal document assistant designed to enhance the efficiency of legal research and analysis. The system utilizes advanced natural language processing (NLP) techniques to process a variety of legal documents, such as text files, Word documents, and PDFs, and provide insightful answers to legal queries.

At its core, this project combines document retrieval and generative models to create a Retrieval-Augmented Generation (RAG) system. It uses vector search with embeddings to identify relevant sections from legal documents and then generates answers by leveraging OpenRouter's GPT model, trained on legal and conversational data.

This tool is aimed at making legal professionals' tasks faster and more accurate by allowing them to query a vast repository of legal information. Whether it’s for lawyers, researchers, or students, "KnowLaw" simplifies the process of extracting useful legal knowledge from extensive document archives.

In [1]:
import os
import requests
from langchain.document_loaders import TextLoader, UnstructuredFileLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from google.colab import files


OPENROUTER_API_KEY = "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"
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"
EMBEDDING_MODEL = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


def load_legal_docs(folder_path):
    all_texts = []
    for file in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file)
        if file.endswith(".txt"):
            loader = TextLoader(full_path)
        elif file.endswith((".docx", ".pdf")):
            loader = UnstructuredFileLoader(full_path)
        else:
            print(f"⚠️ Skipping unsupported file: {file}")
            continue
        documents = loader.load()
        all_texts.extend(documents)
    return all_texts


def build_vector_store(documents):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    split_docs = text_splitter.split_documents(documents)
    vector_store = FAISS.from_documents(split_docs, EMBEDDING_MODEL)
    return vector_store


def retrieve_context(query, vector_store, k=3):
    docs = vector_store.similarity_search(query, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context


def query_openrouter_gpt(prompt):
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "openai/gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    response = requests.post(OPENROUTER_API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"OpenRouter Error {response.status_code}: {response.text}")


def legal_assistant(query, vector_store):
    context = retrieve_context(query, vector_store)
    prompt = f"""You are a helpful legal assistant. Use the following context to answer the user's question:

Context:
{context}

Question: {query}

Answer:"""
    return query_openrouter_gpt(prompt)


if __name__ == "__main__":
    print("📂 Upload your legal documents (.txt, .docx, .pdf)...")
    uploaded = files.upload()

    os.makedirs("legal_docs", exist_ok=True)
    for filename in uploaded.keys():
        new_path = os.path.join("legal_docs", filename)
        os.rename(filename, new_path)

    print("🔍 Loading legal documents...")
    legal_docs = load_legal_docs("legal_docs")
    vs = build_vector_store(legal_docs)

    print("⚖️ Ask your legal question!")
    while True:
        user_question = input("🗨️ Question (type 'exit' to quit): ")
        if user_question.lower() in ["exit", "quit"]:
            break
        try:
            answer = legal_assistant(user_question, vs)
            print(f"\n📜 Answer:\n{answer}\n")
        except Exception as e:
            print(f"❌ Error: {e}")


<ipython-input-1-aa6975f93be0>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  EMBEDDING_MODEL = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

📂 Upload your legal documents (.txt, .docx, .pdf)...


Saving p17027coll5_38487.pdf to p17027coll5_38487.pdf
🔍 Loading legal documents...


<ipython-input-1-aa6975f93be0>:22: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(full_path)


⚖️ Ask your legal question!
🗨️ Question (type 'exit' to quit): explain the case

📜 Answer:
In this case, the juvenile court recognized that the appellant had an interest in developing a relationship with the child, but that this relationship was not parental in nature. The court also expressed concerns regarding the Department of Human Services' lack of efforts in fostering a parental relationship between the appellant and the child. Ultimately, the court ruled that it was not in the best interest of the child to continue with the appellant in place of another individual as a parent. Despite acknowledging the lack of reasonable efforts by DHS, the court did not abuse its discretion in making this decision.

🗨️ Question (type 'exit' to quit): what are the actions taken

📜 Answer:
The actions taken in this case included disestablishing the appellant as the child's legal parent, establishing the child's biological father as the legal parent, and setting aside the voluntary acknowledgment 

In [21]:
!pip install unstructured


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.6/180.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 7.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=bee2e257769442922c435d3044dc3fe0eb1458bc088e86454862155ff32f2545
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [23]:
!pip install "unstructured[docx]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00


In [26]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.4 MB/s eta 0:00:00


In [29]:
!pip install "unstructured[pdf]"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━